In [45]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [46]:
def get_mnist2():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))
    return train_x, train_y

dataset_func = lib.datasets.get_linear3

if True or "train_x" not in globals():
    train_x, train_y = dataset_func()



In [47]:
def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.optimizer.LiveNetOptimizer(net, lr=0.01)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        # optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.SGD, net.parameters(), lr=0.01)
        optimizer = lib.optimizer.optimizer_with_lr_property(torch.optim.Adam, net.parameters(), betas=(0.0, 0.95))
    return optimizer


In [48]:
lib.utils.set_seed()
# network = lib.nets.LINEAR3(l1=0.1)
network = lib.nets.create_livenet_linear3(l1=0.1)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_n
optimizer = create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=30, adaptive_lr=False)


LiveNet


In [53]:

trainer.step(200)


Iˈ0.000 0.563 = 0.204+0.359 lib/trainer.py:89
Iˈ0.052 0.563 = 0.204+0.359 lib/trainer.py:89
Iˈ0.103 0.563 = 0.204+0.360 lib/trainer.py:89
Iˈ0.154 0.563 = 0.204+0.360 lib/trainer.py:89
Iˈ0.206 0.563 = 0.204+0.359 lib/trainer.py:89
Iˈ0.258 0.563 = 0.204+0.359 lib/trainer.py:89
Iˈ0.310 0.563 = 0.204+0.359 lib/trainer.py:89


In [54]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.0000

In [55]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["linear1.weight"][0][1].item()
    return val0
    return param["grads"]["linear1.weight"][0][0].item() / 1000
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

# plt.figure(figsize=(16, 9))
# values = get_param_values(trainer.history, param_picker0)
# plt.plot(values)
# values = get_param_values(trainer.history, param_picker1)
# plt.plot(values)
# plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()

Using matplotlib backend: TkAgg


In [44]:
for name, p in network.named_parameters():
    print(name, f"{p.item():.6f}")

3 -0.308018
4 0.308038
0->3 0.603300
0->4 -0.603251
1->3 -0.615420
1->4 0.615389
2->3 0.603300
2->4 -0.603251
